In [7]:
# example from https://www.youtube.com/watch?v=pDdP0TFzsoQ&list=PLqnslRFeH2UrcDBWF5mfPGpqQDSta6VK4&index=14
# https://github.com/python-engineer/pytorchTutorial/blob/master/14_cnn.py
# PyTorch Tutorial 14 - Convolutional Neural Network (CNN)
# all credits to Patrick Loeber

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [8]:
# Device Config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters
num_epochs = 4
batch_size = 4
learning_rate = 0.001    

In [15]:
# CIFAR10 dataset are PILImage images with a range of [0,1].
# Transform them to tensors of a normalized range of [-1,1].
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))])

# Get Dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# our classes of the dataset
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [16]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # -> n, 3, 32, 32
        x = self.pool(F.relu(self.conv1(x)))  # -> n, 6, 14, 14
        x = self.pool(F.relu(self.conv2(x)))  # -> n, 16, 5, 5
        x = x.view(-1, 16 * 5 * 5)            # -> n, 400
        x = F.relu(self.fc1(x))               # -> n, 120
        x = F.relu(self.fc2(x))               # -> n, 84
        x = self.fc3(x)                       # -> n, 10
        return x


model = ConvNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # origin shape: [4, 3, 32, 32] = 4, 3, 1024
        # input_layer: 3 input channels, 6 output channels, 5 kernel size
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 2000 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

print('Finished Training')
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)

with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')

Epoch [1/4], Step [2000/12500], Loss: 2.2405
Epoch [1/4], Step [4000/12500], Loss: 2.2406
Epoch [1/4], Step [6000/12500], Loss: 2.1737
Epoch [1/4], Step [8000/12500], Loss: 2.1655
Epoch [1/4], Step [10000/12500], Loss: 2.0536
Epoch [1/4], Step [12000/12500], Loss: 2.5109
Epoch [2/4], Step [2000/12500], Loss: 1.8393
Epoch [2/4], Step [4000/12500], Loss: 2.1415
Epoch [2/4], Step [6000/12500], Loss: 1.5724
Epoch [2/4], Step [8000/12500], Loss: 2.5748
Epoch [2/4], Step [10000/12500], Loss: 1.6000
Epoch [2/4], Step [12000/12500], Loss: 1.3468
Epoch [3/4], Step [2000/12500], Loss: 1.6849
Epoch [3/4], Step [4000/12500], Loss: 1.4829
Epoch [3/4], Step [6000/12500], Loss: 2.1452
Epoch [3/4], Step [8000/12500], Loss: 1.4110
Epoch [3/4], Step [10000/12500], Loss: 1.4502
Epoch [3/4], Step [12000/12500], Loss: 1.5788
Epoch [4/4], Step [2000/12500], Loss: 2.9828
Epoch [4/4], Step [4000/12500], Loss: 1.7034
Epoch [4/4], Step [6000/12500], Loss: 1.3719
Epoch [4/4], Step [8000/12500], Loss: 2.7157
Epoc